<a href="https://colab.research.google.com/github/erokemwa/Data-Science/blob/main/price_monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import argparse
import logging
import sys
import time
from typing import Optional, Tuple

import MetaTrader5 as mt5

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

DEFAULT_SYMBOL = "EURUSD"
DEFAULT_PRICE_DELTA = 0.00005
DEFAULT_CHECK_INTERVAL = 5  # seconds
MAX_RETRIES = 3

class TradingBot:
    def __init__(self, symbol: str, price_delta: float, check_interval: int):
        self.symbol = symbol
        self.price_delta = price_delta
        self.check_interval = check_interval
        self.initialized = False

    def initialize_mt5(self) -> bool:
        """Initialize MetaTrader5 with retry logic."""
        for _ in range(MAX_RETRIES):
            if mt5.initialize():
                if mt5.symbol_select(self.symbol, True):
                    self.initialized = True
                    return True
                logger.error("Failed to select symbol %s", self.symbol)
                mt5.shutdown()
            time.sleep(1)
        logger.error("Failed to initialize MT5 after %d attempts", MAX_RETRIES)
        return False

    def get_price_info(self) -> Optional[mt5.Tick]:
        """Fetch price information with error handling."""
        try:
            price_info = mt5.symbol_info_tick(self.symbol)
            if price_info is None:
                logger.warning("No price info received for %s", self.symbol)
            return price_info
        except Exception as e:
            logger.error("Error fetching price info: %s", str(e))
            return None

    def calculate_thresholds(self, starting_price: float) -> Tuple[float, float]:
        """Calculate price thresholds based on initial price."""
        return (
            starting_price + self.price_delta,
            starting_price - self.price_delta
        )

    def monitor_market(self) -> Optional[Tuple[bool, float]]:
        """Main monitoring loop with price threshold checks."""
        if not self.initialized:
            logger.error("MT5 not initialized")
            return None

        price_info = self.get_price_info()
        if not price_info:
            return None

        starting_price = price_info.bid
        upper, lower = self.calculate_thresholds(starting_price)

        logger.info("Starting price: %.5f", starting_price)
        logger.info("Upper threshold: %.5f | Lower threshold: %.5f", upper, lower)

        last_price = starting_price
        try:
            while True:
                price_info = self.get_price_info()
                if not price_info:
                    time.sleep(self.check_interval)
                    continue

                current_price = price_info.bid
                if current_price != last_price:
                    logger.info("Price update: %s = %.5f", self.symbol, current_price)
                    last_price = current_price

                if current_price >= upper:
                    logger.info("Upper threshold breached at %.5f", upper)
                    return True, current_price
                if current_price <= lower:
                    logger.info("Lower threshold breached at %.5f", lower)
                    return False, current_price

                time.sleep(self.check_interval)

        except KeyboardInterrupt:
            logger.info("Monitoring interrupted by user")
            return None

def main():
    parser = argparse.ArgumentParser(description="Market Monitoring Bot")
    parser.add_argument('--symbol', type=str, default=DEFAULT_SYMBOL,
                        help="Financial instrument to monitor")
    parser.add_argument('--delta', type=float, default=DEFAULT_PRICE_DELTA,
                        help="Price delta for threshold calculation")
    parser.add_argument('--interval', type=int, default=DEFAULT_CHECK_INTERVAL,
                        help="Price check interval in seconds")
    args = parser.parse_args()

    bot = TradingBot(
        symbol=args.symbol,
        price_delta=args.delta,
        check_interval=args.interval
    )

    try:
        if not bot.initialize_mt5():
            return

        result = bot.monitor_market()
        if result:
            direction, price = result
            action = "above upper" if direction else "below lower"
            logger.info("Price moved %s limit. Final price: %.5f", action, price)
        else:
            logger.warning("Monitoring stopped without threshold breach")

    except Exception as e:
        logger.exception("Unexpected error occurred: %s", str(e))
    finally:
        mt5.shutdown()
        logger.info("MT5 connection closed")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'MetaTrader5'